In [16]:
import os
import pickle
import PIL.Image
import numpy as np
import pandas as pd
import csv
import sys
from pathlib import Path

# test

import matplotlib.pyplot as plt
%matplotlib inline

# path to stylegan encoder on server
sys.path.insert(1, '/home/shavit/APhotoADay/data/stylegan-encoder')

import dnnlib
import dnnlib.tflib as tflib
import config
# from encoder.generator_model import Generator

from IPython.display import Markdown, display

from IPython.display import Markdown, display

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [17]:
URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'

tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)

generator = Generator(Gs_network, batch_size=1, randomize_noise=False)

NameError: name 'Generator' is not defined

In [18]:
def generate_image(latent_vector, size):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img.resize((size, size))

def show(latent_vector):
    fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
    ax[i].imshow(generate_image(latent_vector))
    ax[i].set_title(latent_vector)
    [x.axis('off') for x in ax]
    plt.show()

In [36]:
#    Given a data set of the form
#    ((original photo, original age, target age), target photo)
#    We take some part of the data set and use it to train a linear model
#    We reserve some of the data to test it afterwards

#    photos should be npy
#    input = lists of the data (should all be the same size and flattened)
#    output = linear regression interpolator
#    note that data can come from multiple people, the more data, the more accurate

def findRegressor(start_photos, start_ages, target_ages, target_photos):

    X = []
    Y = []

    for i in range(len(start_photos)):
        X.append(np.concatenate([start_photos[i], start_ages[i], target_ages[i]], axis=None))
        Y.append(target_photos[i])

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

    regressor = LinearRegression()  
    regressor.fit(X_train, Y_train)

    #print("accuracy = " + str(testModel(X_train, Y_train, regressor)))

    return regressor, X_test, Y_test

#    input = list of data to test on and regressor to test
#    output = accuracy in MSE
def testModel(X, Y, regressor):

    Y_pred = regressor.predict (X)
    return Y_pred

In [20]:
# TESTING FROM HERE DOWN REMOVE THIS DURING USE #

# data1 = []
# data2 = []
# data3 = []

# results = []
# for i in range(1000):
#     data1.append(np.array(np.random.rand()))
#     data2.append(np.array(np.random.rand()))
#     data3.append(np.array(np.random.rand()))
#     results.append(np.array([np.power(data1[i], 1),data2[i], data3[i]]))

# print(np.around(data1, decimals=3))
# print(np.around(data2, decimals=3))
# print(np.around(data3, decimals=3))
# print(np.around(results, decimals=3))

# regressor, X_test, Y_test = findInterpolator(data1, data2, data3, results)

# print (testModel(X_test, Y_test, regressor))

In [43]:
# Receive input

os.chdir("/home/shavit/APhotoADay/True_vs_Learned")

num_of_people = int(input("Enter number of people: "))

paths = []

for person in range(num_of_people):
#     paths.append("regression_data/" + input("Enter path for data for person " + str(person + 1) + ": "))
    paths.append("regression_data/video" + str(person + 1))
    
start_photos = []
start_ages = []
target_ages = []
target_photos = []

first = []
first_age = []

for person in range(num_of_people):
    unsorted_files = os.listdir(paths[person])
    naked_files = sorted(unsorted_files, key=lambda x: int(os.path.splitext(x)[0]))
    
    files = []
    
    for file in naked_files:
        files.append(paths[person] + "/" + file)
    
    first.append(np.load(files[0]).flatten())
    first_age.append(int(Path(files[0]).stem))
    
    for i in range(len(files)):
        start_photos.append(first[person])
        start_ages.append(first_age[person])
        target_ages.append(int(Path(files[i]).stem))
        target_photos.append(np.load(files[i]).flatten())
        
#for i in range (len(start_photos)):
#    print(start_photos[i], start_ages[i], target_ages[i], target_photos[i])
    
regressor, X_test, Y_test = findRegressor(start_photos, start_ages, target_ages, target_photos)

Y_pred = testModel(X_test, Y_test, regressor)
MSE = metrics.mean_squared_error (Y_test, Y_pred)

print (X_test[0].shape)

# New code
age_specific_MSE = []

for i, age in enumerate(Y_pred):
    this_MSE = metrics.mean_squared_error (np.array(Y_pred[i]), np.array(Y_test[i]))
    age_specific_MSE.append([X_test[i][9217], this_MSE])

print(age_specific_MSE)

Enter number of people:  1


(9218,)
[[8000.0, 0.014663009], [7390.0, 0.008961048], [8520.0, 0.010992867], [7570.0, 0.009705851], [8290.0, 0.009262237], [7600.0, 0.009946018], [7120.0, 0.0093971295], [8030.0, 0.008732627], [8350.0, 0.0077429265], [7010.0, 0.007885397], [6990.0, 0.006899232], [8560.0, 0.01037017], [8470.0, 0.008663521], [8700.0, 0.007514693], [8120.0, 0.008629149], [7910.0, 0.0072339824], [7310.0, 0.0082244715], [7870.0, 0.008253856], [8280.0, 0.00816403], [8200.0, 0.0083583705], [7490.0, 0.0101454295], [7770.0, 0.008558638], [7500.0, 0.007643937], [8430.0, 0.009189777], [8610.0, 0.0094704665], [8570.0, 0.0077044484], [7680.0, 0.018287957], [8050.0, 0.008303468], [8650.0, 0.0066871205], [7270.0, 0.009525506], [6980.0, 0.014954459], [8150.0, 0.0071538305], [8620.0, 0.0171627], [7550.0, 0.017385542], [7380.0, 0.009354059], [7200.0, 0.009568165], [8790.0, 0.009241866], [8300.0, 0.010487518]]


In [ ]:
# generate examples


# experiment = "video" + num_of_vid + "_plot_" + str(start_age) + "_to_" + str(end_age) + "_batch=" + str(averaging_ratio) + "_dimensions=" + ("default" if (num_dimensions == 0) else str(num_dimensions)) + "_method=" + ("standard" if (method == 0) else ("puzer" if (method == 1) else "regression"))
# printmd("\n\n**Running experiment: " + experiment + "**", color="blue")

# if not os.path.exists("Results/" + experiment):
#     os.mkdir("Results/" + experiment)

In [32]:
# Video from given picture, using our data (which specific data exactly?)

t_start_photos = []
t_start_ages = []
t_target_ages = []
    
t_first = np.load("regression_data/test/kobi.npy").flatten()
t_first_age = int(input("Enter your age (years): ")) * 365

for i in range (t_first_age, t_first_age * 2, 10):
    t_start_photos.append(t_first)
    t_start_ages.append(t_first_age)
    t_target_ages.append(i)

t_X = []
for i in range(len(t_start_photos)):
    t_X.append(np.concatenate([t_start_photos[i], t_start_ages[i], t_target_ages[i]], axis=None))

results_path = "regression_data/results"

t_Y = regressor.predict(t_X)

if not os.path.exists(results_path):
    os.mkdir(results_path)

for i, prediction in enumerate(t_Y):
    img = generate_image(prediction, 512)
    img.save(results_path + "/" + str(i) + ".png")

KeyboardInterrupt: Interrupted by user

In [19]:
# Display GIFs. This is --NOT-- Python code; It should be run from termianl.


# python3 ../rename3.py . 0

# ffmpeg -f image2 -framerate 75 -pattern_type sequence -start_number 0 -r 2 -i %d.png -vcodec libx264 -acodec aac real_broken.mp4

# ffmpeg -i real_broken.mp4 -c:v libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p real.mp4

# ffmpeg -itsscale 0.027 -i real.mp4 -c copy real_output.mp4

SyntaxError: invalid syntax (<ipython-input-19-2ba3b72d5703>, line 10)

In [ ]:
# Older attempt at displaying GIFs

# def gif_create(delay, path_to_frames, output):
#     os.system("convert -delay " + delay + " -loop 0 " + path_to_frames + "/*.png Results/" + experiment + "/" + output + ".gif")

# gif_create("10", real_data_path, "real")
# gif_create("10", fake_data_path, "fake")